In [42]:
%pip install -Uq upgini catboost

In [43]:
from os.path import exists
import pandas as pd
df_path="/train.csv"
df=pd.read_csv(df_path)
df=df.sample(n=7_500, random_state=0)
df["store"]=df["store"].astype(str)
df["date"]=pd.to_datetime(df["date"])
df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()


,date,store,item,sales
0,2013-01-01,3,27,11
1,2013-01-01,10,21,33
2,2013-01-01,5,24,26
3,2013-01-02,1,13,31
4,2013-01-02,6,40,9


In [44]:
from datetime import date
train= df[df["date"]<"2017-01-01"]
test= df[df["date"]>="2017-01-01"]
train_features= train.drop(columns=["sales"])
train_target= train["sales"]
test_features= test.drop(columns=["sales"])
test_target= test["sales"]

In [45]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType
enricher= FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv=CVType.time_series
)

In [46]:
enricher.fit(train_features,train_target,eval_set=[(test_features,test_target)])

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION




Column name,Status,Errors
date,All valid,-
target,All valid,-



Running search request, search_id=1f30af1e-5029-4fa8-b3bc-bfd26a6abf27
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


46 relevant feature(s) found with the search keys: ['date_0e8763']


f_weather_date_weather_pca_0_d7e0a1fc,0.0939,100.0000,"-28.0304, 18.5738, -26.7633",Upgini,Weather & climate normals data,Daily
f_weather_date_weather_umap_33_89bb7578,0.0494,100.0000,"4.7908, 4.818, 5.009",Upgini,Weather & climate normals data,Daily
f_weather_date_weather_umap_31_fa6d9a99,0.0293,100.0000,"5.141, 5.3038, 5.0481",Upgini,Weather & climate normals data,Daily
f_weather_date_weather_umap_48_b39cd0c4,0.0276,100.0000,"5.1958, 5.8361, 5.1545",Upgini,Weather & climate normals data,Daily
f_events_date_year_cos1_9014a856,0.0261,100.0000,"-0.9937, 0.9524, -0.6584",Upgini,Calendar data,Daily
f_autofe_mul_0cc09d2d,0.0253,100.0000,"29.7593, -5.7651, -37.2002",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_25296268,0.0243,100.0000,"-1.1127, 0.6537, -0.6974",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_a66c2b64,0.0154,100.0000,"-0.0003, -0.0004, -0.0003",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_3e22df83,0.0136,100.0000,"-0.0242, -0.0083, 0.0175",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_26644ad1,0.0108,100.0000,"0.9167, 0.9487, 0.8132",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_de088df4,0.0106,100.0000,"-0.3981, -0.2841, -0.6635",Upgini,AutoFE: features from Calendar data,Daily


Upgini,Weather & climate normals data,0.2088,10
Upgini,"AutoFE: features from Calendar data,Markets data",0.1136,16
Upgini,AutoFE: features from Markets data,0.0363,10
Upgini,Calendar data,0.0263,3
Upgini,AutoFE: features from Calendar data,0.0176,3
Upgini,Markets data,0.0002,2
Upgini,World economic indicators,0.0002,2


"Calendar data,Markets data",f_autofe_mul_0cc09d2d,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_1f195998,*
"Calendar data,Markets data",f_autofe_mul_25296268,f_events_date_week_sin1_847b5db1,f_financial_date_vix_7d_to_1y_634c77eb,*
"Calendar data,Markets data",f_autofe_div_a66c2b64,f_events_date_week_sin1_847b5db1,f_financial_date_snp500_fdb64447,/
"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/
Markets data,f_autofe_mul_26644ad1,f_financial_date_snp500_7d_to_1y_378b9f99,f_financial_date_usd_eur_5d1b9c8f,*
Calendar data,f_autofe_mul_de088df4,f_events_date_year_cos2_7acbaf1b,f_events_date_year_cos3_0570af1d,*
"Calendar data,Markets data",f_autofe_mul_51efd300,f_events_date_week_sin1_847b5db1,f_financial_date_snp500_fdb64447,*
Markets data,f_autofe_div_cc3842ef,f_financial_date_crude_oil_1f195998,f_financial_date_snp500_fdb64447,/
Calendar data,f_autofe_div_ba625fe9,f_events_date_week_sin1_847b5db1,f_events_date_year_cos1_9014a856,/
"Calendar data,Markets data",f_autofe_div_cf5dd7be,f_events_date_week_sin1_847b5db1,f_financial_date_vix_7d_to_1y_634c77eb,/
"Calendar data,Markets data",f_autofe_div_0c0eef24,f_events_date_week_cos3_7525fe31,f_financial_date_gold_7d_to_7d_1y_shift_a8127a43,/



Examples of outliers with maximum value of target:
17    196
34    176
6     171
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,5997,50.4912,332.7606,280.2804,52.4802
Eval 1,1503,59.8496,549.7791,470.9867,78.7924


In [47]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric
model= CatBoostRegressor
model= CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)
enricher.calculate_metrics(
  train_features,train_target,eval_set=[(test_features,test_target)],
 estimator= model,scoring="mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,5997,50.4912,0.259327,0.218996,0.040331
1,Eval 1,1503,59.8496,0.269249,0.208061,0.061188


In [48]:
enriched_train_features=enricher.transform(train_features, keep_input=True)
enriched_test_features=enricher.transform(test_features, keep_input=True)

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 3173 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=6cfac064-a5d4-41c3-b796-dac2afae1bbf
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 4613 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=d8d1d148-69d2-47df-aac7-3954f3f889d2
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


In [49]:
enriched_train_features.head()

,date,store,item,f_weather_date_weather_pca_0_d7e0a1fc,f_weather_date_weather_umap_33_89bb7578,f_weather_date_weather_umap_31_fa6d9a99,f_weather_date_weather_umap_48_b39cd0c4,f_events_date_year_cos1_9014a856,f_autofe_mul_0cc09d2d,f_autofe_mul_25296268,...,f_autofe_div_8c2461d8,f_weather_date_weather_umap_12_d03be9a0,f_economic_date_cci_pca_5_b433b3c8,f_economic_date_cci_pca_6_aa7c1005,f_events_date_week_cos3_7525fe31,f_events_date_week_sin1_847b5db1,f_financial_date_finance_pca_1_38f83e84,f_financial_date_usd_daydiff_8bcdeec4,f_weather_date_weather_pca_34_a2bb42b3,f_weather_date_weather_umap_35_5ddaa0ba
0,2013-01-01,3,27,29.676683,4.644803,4.712653,4.540985,0.985220,90.462910,0.899029,...,427.084435,3.739055,1.573056,-0.472399,-0.900969,0.781831,-1.690329,0.149994,-2.431882,4.767730
1,2013-01-01,10,21,29.676683,4.644803,4.712653,4.540985,0.985220,90.462910,0.899029,...,427.084435,3.739055,1.573056,-0.472399,-0.900969,0.781831,-1.690329,0.149994,-2.431882,4.767730
2,2013-01-01,5,24,29.676683,4.644803,4.712653,4.540985,0.985220,90.462910,0.899029,...,427.084435,3.739055,1.573056,-0.472399,-0.900969,0.781831,-1.690329,0.149994,-2.431882,4.767730
3,2013-01-02,1,13,29.035729,4.641315,4.711289,4.573898,0.982126,90.178773,1.123414,...,NaN,3.723387,1.573056,-0.472399,0.623490,0.974928,-0.152911,0.000000,-3.076863,4.755714
4,2013-01-02,6,40,29.035729,4.641315,4.711289,4.573898,0.982126,90.178773,1.123414,...,NaN,3.723387,1.573056,-0.472399,0.623490,0.974928,-0.152911,0.000000,-3.076863,4.755714


In [50]:
model.fit(train_features,train_target)
preds= model.predict(test_features)
eval_metric(test_target.values,preds,"SMAPE")

[41.43490634616988]

In [51]:
model.fit(enriched_train_features,train_target)
e_preds= model.predict(enriched_test_features)
eval_metric(test_target.values,e_preds,"SMAPE")

[16.030969725506356]